In [57]:
import numpy as np
import aocd
from scipy.spatial import distance
import pandas as pd
import itertools

In [58]:
test_data = """7,1
11,1
11,7
9,7
9,5
2,5
2,3
7,3"""

In [59]:
def parse_data(data):
    return np.array([[int(x) for x in line.split(",")] for line in data.split("\n")])

def solve_part_1(data):
    array = parse_data(data)
    distance_matrix = distance.pdist(array, metric="cityblock")
    indices = itertools.combinations(range(len(array)), 2)
    df = pd.DataFrame(data = {"dists": distance_matrix, "indices": indices})
    df = df.sort_values("dists", ascending=False)
    furthest = df.iloc[0].indices
    c1 = array[furthest[0]]
    c2 = array[furthest[1]]
    width = np.max([c1[0], c2[0]]) - np.min([c1[0], c2[0]]) +1
    height = np.max([c1[1], c2[1]]) - np.min([c1[1], c2[1]]) +1
    print(f"The largest rectangle with red corners is {width*height} tiles, at {c1, c2}")
    return df
    

In [60]:
test_array = parse_data(test_data)

In [61]:
df_test = solve_part_1(test_data)

The largest rectangle with red corners is 50 tiles, at (array([11,  1]), array([2, 5]))


In [62]:
data = aocd.get_data()

In [63]:
df = solve_part_1(data)

The largest rectangle with red corners is 4786902990 tiles, at (array([85835, 83550]), array([16266, 14744]))


## Part Two

In [64]:
from skimage.draw import polygon
import tqdm

In [65]:
#create the array:
def make_floor(data):
    coords = parse_data(data)
    # Determine array size (max coordinate + 1)
    max_x = coords[:, 1].max() + 2   # row index
    max_y = coords[:, 0].max() + 2   # column index

    print(f"Creating an array of {max_x, max_y} ")

    # Create 2D array
    arr = np.empty((max_x, max_y), dtype=np.uint8)
    # arr[::] = 0

    print(f"Created array of size {arr.shape}")

    # use skimage to make the polygon between all the read corners
    r = coords[:,1]
    c = coords[:,0]
    rr, cc = polygon(r,c)

    arr[rr,cc] = 1
    # Set the corners
    for x,y in coords:
        arr[y,x] = 1
    
    # print(arr)
    return coords, arr

In [66]:
coords, floor = make_floor(test_data)

Creating an array of (np.int64(9), np.int64(13)) 
Created array of size (9, 13)


In [67]:
# coords, floor = make_floor(data)

In [68]:
def solve_part_2(data):
    df = solve_part_1(data)
    coords, floor = make_floor(data)
    print(f"We are going to check {len(df)} corner pairs...")
    for i, r in tqdm.tqdm(df.iterrows()):
        indices = r.indices
        c1 = test_array[indices[0]]
        c2 = test_array[indices[1]]
        # print(c1, c2)
        x1, y1 = c1
        x2, y2 = c2
        x_min, x_max = sorted([x1, x2])
        y_min, y_max = sorted([y1,y2])
        # print(x_min, x_max, y_min, y_max)
        floor_slice = floor[y_min:y_max+1, x_min: x_max+1]
        if floor_slice.sum() == floor_slice.size:
            print(f"largest rectangle we can make is at {c1, c2}, area is {floor_slice.size}")
            
            break
        

In [69]:
solve_part_2(test_data)

The largest rectangle with red corners is 50 tiles, at (array([11,  1]), array([2, 5]))
Creating an array of (np.int64(9), np.int64(13)) 
Created array of size (9, 13)
We are going to check 28 corner pairs...


7it [00:00, 1020.62it/s]

largest rectangle we can make is at (array([9, 5]), array([2, 3])), area is 24


In [70]:
#Dont run this on data. it is waaay slow creating a 10billion pixel array

In [71]:
coords = parse_data(data)

In [113]:
from shapely import Polygon

In [137]:
def solve_part_2_2(data):
    df = solve_part_1(data)
    coords = parse_data(data)
    polygon = Polygon(coords)
    print(f"We are going to check {len(df)} corner pairs...")
    for i, r in tqdm.tqdm(df.iterrows()):
        indices = r.indices
        c1 = coords[indices[0]].astype(int)
        c2 = coords[indices[1]].astype(int)
        x1, y1 = c1
        x2, y2 = c2
        x_min, x_max = sorted([x1, x2])
        y_min, y_max = sorted([y1,y2])
        c1 = (x_min, y_min)
        c2 = (x_max, y_min)
        c3 = (x_max, y_max)
        c4 = (x_min, y_max)
        rect = Polygon(np.vstack([c1,c2,c3,c4]))
        if polygon.contains(rect):
            print(f"largest rectangle we can make is at corner pair {i}, area is {(x_max-x_min + 1) * (y_max-y_min+1)}")
            break

In [138]:
solve_part_2_2(test_data)

The largest rectangle with red corners is 50 tiles, at (array([11,  1]), array([2, 5]))
We are going to check 28 corner pairs...


7it [00:00, 2422.05it/s]

largest rectangle we can make is at corner pair 23, area is 24


In [139]:
solve_part_2_2(data)

The largest rectangle with red corners is 4786902990 tiles, at (array([85835, 83550]), array([16266, 14744]))
We are going to check 122760 corner pairs...


35144it [00:07, 4627.75it/s]

largest rectangle we can make is at corner pair 84009, area is 1571016172
